In [1]:
import os
from fnmatch import fnmatch
import numpy as np
from PIL import Image
from filtering.filters import Median
from feature_extraction.glcm import Glcm
import sklearn
import time
from gmm import Gmm
from sklearn import preprocessing
import pickle

In [2]:
#Read features of training samples
train_samples_feats_matrix_GLCM = np.load("train_samples_feats_matrix_GLCM.npy")

In [ ]:
#Load features of validation samples Gabor
valid_samples_feats_matrix_GLCM = np.load("valid_samples_feats_matrix_GLCM.npy")

In [3]:
test__samples_feats_matrix_GLCM = np.load("test_samples_feats_matrix_GLCM.npy")

In [ ]:
#Train a gaussian mixture model from training data
t0 = time.time()
gm_model = sklearn.mixture.GaussianMixture(n_components=100, covariance_type='spherical')
gm_model.fit(train_samples_feats_matrix_GLCM)
print("traing time: ", time.time() - t0)

In [4]:
gm_model = pickle.load( open( 'GMM_GLCM_2000', "rb" ))

In [5]:
#Predict labels of the new data points
y_predict = gm_model.predict(test__samples_feats_matrix_GLCM)
#Predict novelty score of new data points
y_predict_score = gm_model.score_samples(test__samples_feats_matrix_GLCM)
tmp = []

x_predict_score = gm_model.score_samples(train_samples_feats_matrix_GLCM)
novelty_thresh = np.amin(x_predict_score)

# if data point belongs to the model then 1 else 0
tmp = np.repeat(1, 10000)
tmp[y_predict_score < 183.164696046] = 0


In [6]:
#Ground truth
y1 = np.repeat(1, 6000) #normal
y2 = np.repeat(0, 4000) #abnormal
y = np.concatenate((y1,y2))

In [7]:
f1_binary = sklearn.metrics.f1_score(y, tmp, pos_label =0, average = 'binary')
f1_macro = sklearn.metrics.f1_score(y, tmp, average = 'macro')
auc = sklearn.metrics.roc_auc_score(y, y_predict_score)
Math_Cof = sklearn.metrics.matthews_corrcoef(y, tmp) 
tn, fp, fn, tp  = sklearn.metrics.confusion_matrix(y, tmp).ravel()
print ("f1_binary: ", f1_binary, "f1_macro: ", f1_macro, "MAth_cof: ", Math_Cof , " auc: ", auc)
print("confusion matrix", tn, fp, fn, tp)

('f1_binary: ', 0.58061480552070255, 'f1_macro: ', 0.42137804293695169, 'MAth_cof: ', 0.12426860077916524, ' auc: ', 0.58710925000000003)
('confusion matrix', 3702, 298, 5050, 950)


In [ ]:
#Get different thresholds value from ROC with corresponding F1_score and AUC
fpr,tpr,thresh = sklearn.metrics.roc_curve(y, y_predict_score)
for t, thres in enumerate (thresh) :
    tmp = np.repeat(1,10000)
    tmp[y_predict_score < thres] = 0
    f1 = sklearn.metrics.f1_score(y, tmp,pos_label=0, average = 'binary')
    print ("f1: ", f1, "thres: ", thres)

In [ ]:
import matplotlib.pyplot as pl
import matplotlib
from skimage import io

fpr,tpr,thresh = sklearn.metrics.roc_curve(y,y_predict_score)

f,ax = pl.subplots(1,1)
ax.plot(fpr,tpr,label="GMM")
ax.set_xlabel("False Positive Rate")
ax.set_ylabel("True Positive Rate")
ax.set_title("ROC curves")
ax.legend(loc="lower right")
# these are matplotlib.patch.Patch properties
props = dict(boxstyle='square', facecolor='white', alpha=0.5)

textstr = '$F1-binary=%.2f$\n$F1-macro=%.2f$\n$math-corcoeffient=%.2f$\n$AUC=%.2f$'%(0.58, 0.63, 0.27, 0.68)
# place a text box in upper left in axes coords
ax.text(0.95, 0.2, textstr, transform=ax.transAxes, fontsize=14,
    horizontalalignment='right', verticalalignment='bottom', bbox=props)

io.show()


In [ ]:
f.savefig('GMM_GLCM_2000.png')